In [1]:
from BaseEncoder import BaseEncoder
from DataLoader import read_examples_from_file, ReviewDataset

from transformers import AutoTokenizer, TFAutoModel
import torch
from torch import nn
from torch.utils.data import DataLoader

from SpanMltri import SpanMltri

Using cuda device


# Test Base Encoder

In [2]:
sentence = 'Aku seorang kapiten'

In [3]:
base_encoder = BaseEncoder()

In [4]:
sample_tokenized = base_encoder.tokenize(['Aku', 'seorang', 'kapiten'])

In [5]:
sample_tokenized.shape

torch.Size([40])

# Data Loader

In [6]:
FILE_PATH = "dataset/test_2k_output_Feb_26_2020_output_Feb_18_2021.tsv"

In [7]:
test_data = ReviewDataset(FILE_PATH)

In [8]:
len(test_data)

200

# Test Model

In [9]:
span_mltri = SpanMltri()

In [11]:
%%time 

BATCH_SIZE = 8
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(span_mltri.parameters(), lr=0.005)

lambda_t = 0.5
lambda_r = 0.5

size = len(test_dataloader.dataset)
for batch, X in enumerate(test_dataloader):
    current_te_label_list = test_data.te_label_list[(batch)*BATCH_SIZE:(batch+1)*BATCH_SIZE]
    current_relations = test_data.relations[(batch)*BATCH_SIZE:(batch+1)*BATCH_SIZE]
    
    logits_term_scorer, span_ranges, logits_relation_scorer, span_pair_ranges = span_mltri(X)
    
    y_te_true = []
    CURRENT_BATCH_SIZE = min(len(current_te_label_list), BATCH_SIZE)
    for i in range(CURRENT_BATCH_SIZE):
        y_ = []
        for span_range in span_ranges:
            if span_range in current_te_label_list[i]:
                label = current_te_label_list[i][span_range]
                if label == 'ASPECT':
                    y_.append(1)
                elif label == 'SENTIMENT':
                    y_.append(2)
            else: # label is O
                y_.append(0)        
        y_te_true.append(torch.Tensor(y_))
    y_te_true = torch.stack(y_te_true)
    y_te_true = y_te_true.to(torch.long)
    
    logits_term_scorer = logits_term_scorer.reshape(logits_term_scorer.shape[0]*logits_term_scorer.shape[1], logits_term_scorer.shape[-1])
    y_te_true = y_te_true.reshape(-1)
    te_loss = loss_fn(logits_term_scorer, y_te_true)
    
    y_paote_true = []
    CURRENT_BATCH_SIZE = min(len(current_relations), BATCH_SIZE)
    for i in range(CURRENT_BATCH_SIZE):
        y_ = []
        for span_pair_range in span_pair_ranges[i]:
            if span_pair_range not in current_relations[i]:
                y_.append(0)
            else:
                label = current_relations[i][span_pair_range]
                if label == 'PO':
                    y_.append(1)
                elif label == 'NG':
                    y_.append(2)
                elif label == 'NT':
                    y_.append(3)
        y_paote_true.append(torch.Tensor(y_))
    y_paote_true = torch.stack(y_paote_true)
    y_paote_true = y_paote_true.to(torch.long)
    
    logits_relation_scorer = logits_relation_scorer.reshape(logits_relation_scorer.shape[0]*logits_relation_scorer.shape[1], logits_relation_scorer.shape[-1])
    y_paote_true = y_paote_true.reshape(-1)
    paote_loss = loss_fn(logits_relation_scorer, y_paote_true)
    
    total_loss = lambda_t*te_loss + lambda_r*paote_loss
    
    # Backpropagation
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if batch % 2 == 0:
        total_loss, current = total_loss.item(), batch * len(X)
        print(f"loss: {total_loss:>7f}  [{current:>5d}/{size:>5d}]")

otw backprop
done backprop
loss: 0.876627  [    0/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 1.075147  [   16/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 1.073519  [   32/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946236  [   48/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946233  [   64/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946175  [   80/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946003  [   96/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946404  [  112/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946118  [  128/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.945775  [  144/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.946061  [  160/  200]
otw backprop
done backprop
otw backprop
done backprop
loss: 0.945832  [  176/  200]
otw backprop
done b

In [12]:
print("Model structure: ", span_mltri, "\n\n")

for name, param in span_mltri.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  SpanMltri(
  (base_encoder): BaseEncoder(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31923, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias